## LSTM class Developed
-----------------------------
[feat] 7/16
- ! to prevent overfitting use K-fold, validation split ! <- NEED TO DEVELOP
- basical LSTM model => lstm dense need to doubled
- not tested
- without AOC, interest rate
- bidirectional LSTM Model

[feat] 7/17
- teseted quality bad
- need to develop k-fold
- need to develop bidirectional
- need to postscale


imorts & get datas
------------

In [ ]:
#!pip install -U finance-datareader

#%pip install FinanceDataReader

In [70]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
#import FinanceDataReader as fdr

In [71]:
data=pd.read_csv('/content/drive/MyDrive/2023_1st_vac/KRX_DATA/train.csv')

In [72]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목명",inplace=False)
subject_Data.head()
subject_Data.columns

pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])

divided = pv_s_data.index.get_level_values('종목코드').unique()

sliced_dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    sliced_dataframes.append(sliced_dataframe)

In [73]:
data=sliced_dataframes
data

[           일자     거래량     고가     시가     저가     종가
 0    20210601  114966  14700  14700  14450  14600
 1    20210602  109559  14700  14700  14450  14500
 2    20210603   96158  14650  14550  14450  14600
 3    20210604  133900  14800  14600  14550  14700
 4    20210607  511140  15550  14800  14750  15150
 ..        ...     ...    ...    ...    ...    ...
 489  20230523  641524   9900   9160   9160   9770
 490  20230524  205243   9820   9770   9550   9740
 491  20230525  398326  10180   9660   9660  10040
 492  20230526  196257  10150  10050   9850   9850
 493  20230530  201361  10040   9960   9640   9700
 
 [494 rows x 6 columns],
            일자      거래량    고가    시가    저가    종가
 0    20210601   480096  1190  1180  1155  1175
 1    20210602   928675  1225  1190  1180  1210
 2    20210603   508487  1230  1220  1190  1200
 3    20210604   445565  1200  1200  1175  1195
 4    20210607   769433  1220  1220  1190  1210
 ..        ...      ...   ...   ...   ...   ...
 489  20230523   638696  

In [74]:
data[0].shape

(494, 6)

In [75]:
class LSTM_Object():

  def __init__(self,data):
    #get sliced data
    self.data=data

  def modeling(self):

    self.prescale()
    self.model=Sequential()
    #layers
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(Dropout(0.25))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    self.model.add(Dropout(0.25))
    self.model.add(LSTM(1024,return_sequences=True,input_shape=(self.x_train.shape[1], self.x_train.shape[2])))
    #output layer
    self.model.add(Dense(1, activation='sigmoid'))

    self.filename = join('tmp', 'ckeckpointer.ckpt')
    checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True,monitor='val_loss',verbose=1)

    earlystopping = EarlyStopping(monitor='val_loss', patience=100)


    self.model.compile(loss = 'binary_crossentropy', optimizer=Adam(learning_rate=0.001))
    history=self.model.fit(self.x_train,self.y_train,epochs=10000,batch_size=128,validation_data=(self.x_train,self.y_train),shuffle=False,callbacks=[checkpoint, earlystopping])


  def prescale(self):
    x_scaler=MinMaxScaler()
    self.y_scaler=MinMaxScaler()
    self.data[['거래량', '고가', '시가', '저가']]=x_scaler.fit_transform(self.data[['거래량', '고가', '시가', '저가']])
    self.data['종가']=self.y_scaler.fit_transform(self.data['종가'].values.reshape(-1,1))
    x_data=self.data[['거래량', '고가', '시가', '저가']]
    y_data=self.data['종가']
    x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2)

    ### reshape input datas()

    input_dim = constant(2)
    self.x_train=expand_dims(x_train,input_dim)
    self.x_test=expand_dims(x_test,input_dim)
    self.y_train=y_train
    self.y_test=y_test



  def ret_learning(self):
    # rescale to get real value
    rescaled_real = self.y_scaler.inverse_transform(np.array(self.y_test).reshape(-1,1))

    self.prescale()
    self.modeling()
    self.model.load_weights(self.filename)
    pred = self.model.predict(self.x_test)
    rescaled_pred=self.y_scaler.inverse_transform(np.array(pred).reshape(-1,1))


    plt.figure(figsize=(12, 9))
    plt.plot(np.asarray(rescaled_real), label='actual')
    plt.plot(pred, label='prediction')
    plt.legend()
    plt.show()

In [76]:
test_1=data[0]

In [77]:
L=LSTM_Object(test_1)
L.prescale()

In [78]:
L.x_train.shape[1]


4

In [79]:
L.x_train.shape[2]


1

In [ ]:
L.ret_learning()

Epoch 1/10000
3/4 [=====================>........] - ETA: 0s - loss: 0.6873
Epoch 1: val_loss improved from inf to 0.65428, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 19s 3s/step - loss: 0.6870 - val_loss: 0.6543
Epoch 2/10000
3/4 [=====================>........] - ETA: 0s - loss: 0.6507
Epoch 2: val_loss improved from 0.65428 to 0.63562, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 5s 2s/step - loss: 0.6518 - val_loss: 0.6356
Epoch 3/10000
3/4 [=====================>........] - ETA: 0s - loss: 0.6397
Epoch 3: val_loss did not improve from 0.63562
4/4 [==============================] - 0s 117ms/step - loss: 0.6404 - val_loss: 0.6445
Epoch 4/10000
3/4 [=====================>........] - ETA: 0s - loss: 0.6405
Epoch 4: val_loss improved from 0.63562 to 0.61284, saving model to tmp/ckeckpointer.ckpt
4/4 [==============================] - 4s 1s/step - loss: 0.6406 - val_loss: 0.6128
Epoch 5/10000
3/4 [=====================>..